<a href="https://colab.research.google.com/github/juvi-160/YBI-Internship/blob/main/Big_Data_Specialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b7410a665884274e792604efe1a7dc23cebb75b412bb855d53a2cf540f845ae7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


PySpark MLib

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
#import Data from github
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
df = load_iris(as_frame = True)
df = df.frame
#df = pd.read_csv('https://github.com/YBI-Foundation/Dataset/raw/main/IRIS.csv')
df = spark.createDataFrame(df)

In [ ]:
df.show()

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

In [ ]:
df.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- target: long (nullable = true)



In [ ]:
df.columns

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)',
 'target']

Vector assembler

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
featureassembler = VectorAssembler(inputCols = ['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)'], outputCol = 'Features')

In [ ]:
output = featureassembler.transform(df)

In [ ]:
output.show()

+-----------------+----------------+-----------------+----------------+------+-----------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|         Features|
+-----------------+----------------+-----------------+----------------+------+-----------------+
|              5.1|             3.5|              1.4|             0.2|     0|[5.1,3.5,1.4,0.2]|
|              4.9|             3.0|              1.4|             0.2|     0|[4.9,3.0,1.4,0.2]|
|              4.7|             3.2|              1.3|             0.2|     0|[4.7,3.2,1.3,0.2]|
|              4.6|             3.1|              1.5|             0.2|     0|[4.6,3.1,1.5,0.2]|
|              5.0|             3.6|              1.4|             0.2|     0|[5.0,3.6,1.4,0.2]|
|              5.4|             3.9|              1.7|             0.4|     0|[5.4,3.9,1.7,0.4]|
|              4.6|             3.4|              1.4|             0.3|     0|[4.6,3.4,1.4,0.3]|
|              5.0|           

Data Model for Features


In [ ]:
modeldata = output.select('Features','target')

splitting we have to use random split method

In [ ]:
train_data,test_data = modeldata.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes(featuresCol = 'Features', labelCol = 'target')

In [ ]:
nb = nb.fit(train_data)

In [ ]:
y_pred = nb.transform(test_data)

Accuracy to learn this we have to learn GroupBy function

In [ ]:
y_pred.groupBy('target','prediction').count().show()


+------+----------+-----+
|target|prediction|count|
+------+----------+-----+
|     0|       0.0|    7|
|     1|       1.0|    6|
|     1|       2.0|    4|
|     2|       2.0|    7|
+------+----------+-----+



In [ ]:
from sklearn.metrics import confusion_matrix
pred = y_pred.select('prediction').collect()
org = y_pred.select('target').collect()
cm = print(confusion_matrix(org,pred))
cm

[[7 0 0]
 [0 6 4]
 [0 0 7]]


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'target', predictionCol = 'prediction')

In [ ]:
accuracy = evaluator.evaluate(y_pred)

In [ ]:
accuracy

0.8310185185185186

DataBrick is Cloud Services. Notebook style coding is much prefered nowadays.